In [1]:
import cv2
import numpy as np
import dlib
import sys
from scipy.spatial import Delaunay
import imageio

#function to morph a single triangle
def morph(img1, img2, img, t1, t2, t, alpha):
    r = cv2.boundingRect(np.float32([t]))
    tRect = [(t[i][0] - r[0], t[i][1] - r[1]) for i in range(3)]
    #create a mask for the triangle
    mask = np.zeros((r[3], r[2], 3), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0))
    size = (r[2], r[3])
    #creating a new image by warping the source image
    warpImage1 = cv2.warpAffine(img1, cv2.getAffineTransform(np.float32(t1), np.float32(tRect)), size)
    warpImage2 = cv2.warpAffine(img2, cv2.getAffineTransform(np.float32(t2), np.float32(tRect)), size)
    #blend the warped image to create final morphed image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = cv2.addWeighted(warpImage1, 1.0 - alpha, warpImage2, alpha, 0.0)

#creating face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#loading source and target image
img2=cv2.imread("trump.jpg")
img1=cv2.imread("kamla.jpg")
max_width = max(img1.shape[1], img2.shape[1])
max_height = max(img1.shape[0], img2.shape[0])

# Resize the images to the same size
img1 = cv2.resize(img1, (max_width, max_height))
img2 = cv2.resize(img2, (max_width, max_height))

#converting image into grayscale
gray1 = cv2.cvtColor(src=img1, code=cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(src=img2, code=cv2.COLOR_BGR2GRAY)

#creating landmark points
faces = detector(gray1)
landmarks = [predictor(image=gray1, box=face) for face in faces]
landmark_points_1 = [(landmarks[i].part(n).x, landmarks[i].part(n).y) for i in range(len(landmarks)) for n in range(68)]
h, w = img1.shape[:2]
landmark_points_1 += [(0, 0), (0, h-1), (0, h//2), (w-1, 0), (w//2, 0), (w-1, h-1), (w-1, h//2-1), (w//2-1, h-1)]


#creating landmark points
faces = detector(gray2)
landmarks = [predictor(image=gray2, box=face) for face in faces]
landmark_points_2 = [(landmarks[i].part(n).x, landmarks[i].part(n).y) for i in range(len(landmarks)) for n in range(68)]
h, w = img2.shape[:2]
landmark_points_2 += [(0, 0), (0, h-1), (0, h//2), (w-1, 0), (w//2, 0), (w-1, h-1), (w-1, h//2-1), (w//2-1, h-1)]



points_1 = np.array(landmark_points_1,np.int32)
convexhull_1 = cv2.convexHull(points_1)
points_2 = np.array(landmark_points_2,np.int32)
convexhull_2 = cv2.convexHull(points_2)


#computing the delaunay triangle
tri = Delaunay(landmark_points_1)
triangle_indexes = tri.simplices

imgMorph = []
#varing alpha values
for alpha in np.linspace(0, 1, 200):
    points = (1 - alpha) * points_1 + alpha * points_2
    tempimg = np.zeros_like(img1)
    for triangle in triangle_indexes:
        morph(img1, img2, tempimg, points_1[triangle], points_2[triangle], points[triangle], alpha)
    imgMorph.append(cv2.cvtColor(tempimg, cv2.COLOR_BGR2RGB))

# Save output as GIF
imageio.mimwrite('output.gif', imgMorph, fps=30)



